In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [2]:
df = pd.read_csv('data/raw.csv')

In [3]:
df.head()

,_id,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,64f61b3e5bd88f2233bf5ded,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,64f61b3e5bd88f2233bf5dee,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,64f61b3e5bd88f2233bf5def,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,64f61b3e5bd88f2233bf5df0,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,64f61b3e5bd88f2233bf5df1,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [4]:
df.drop(columns=['_id','Transaction ID','Date','Customer ID'], inplace=True)

In [5]:
y = df['Total Amount']
X = df.drop(columns='Total Amount')

In [8]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('StandardScaler', num_transformer, num_features ),
        ('OneHotEncoder', cat_transformer, cat_features),
    ]
)

In [9]:
X = preprocessor.fit_transform(X)

In [11]:
X.shape

(1000, 8)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape


((800, 8), (200, 8))

In [13]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [14]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 215.8301
- Mean Absolute Error: 175.3200
- R2 Score: 0.8537
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 205.2436
- Mean Absolute Error: 172.9100
- R2 Score: 0.8561


Lasso
Model performance for Training set
- Root Mean Squared Error: 215.2713
- Mean Absolute Error: 176.2541
- R2 Score: 0.8545
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 204.5791
- Mean Absolute Error: 173.0144
- R2 Score: 0.8570


Ridge
Model performance for Training set
- Root Mean Squared Error: 215.2441
- Mean Absolute Error: 176.1744
- R2 Score: 0.8545
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 204.6626
- Mean Absolute Error: 173.1972
- R2 Score: 0.8569


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 77.1630
- Mean Absolute Error: 43.3650
- R2 Score: 0.98

In [15]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
4,Decision Tree,1.000000
5,Random Forest Regressor,1.000000
6,XGBRegressor,1.000000
7,CatBoosting Regressor,0.999976
3,K-Neighbors Regressor,0.964732
8,AdaBoost Regressor,0.950974
1,Lasso,0.857026
2,Ridge,0.856909
0,Linear Regression,0.856096
